In [2]:
#загрузка необходимых модулей и библиотек

import pandas as pd
import csvtools 
import pymorphy2 
import random
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter 
import prozhito_tools.dump

from string import punctuation as punct
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
 
import nltk
nltk.download("stopwords")
from pymorphy2 import MorphAnalyzer 
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()

from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

# источник дампа: https://www.dropbox.com/sh/8vfjjt8107sv9r3/AADOVR795MxFovpuGN9PT_JZa?dl=0

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/laidhimonthegreen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def normalize(text):
    """ нормализация текста """
    words = text.lower().split()
    words = [word.strip(punct) for word in words]
    words = [word for word in words if word and word not in stops] 
    return words

In [36]:
# задаем "кириллические" символы
cyr = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя " 
cyrillic = cyr + cyr.upper() + punct

def is_russian(text):
    """ определяем, кириллический ли текст """
    all_len = len(text)
    if all_len:
        cyr_len = 0
        for sym in text:
            if sym in cyrillic:
                cyr_len += 1
        if cyr_len/all_len > 0.5:
            if ("ї" not in text) and ("є" not in text) and ("ѣ" not in text):
                return True
    return False

In [37]:
def is_poetry(text):
    """ проверяем, есть ли в тексте длинные стихи """
    lines = text.split("\n")
    poetry_lines = [x for x in lines if x.strip().startswith(">")]
    if len(poetry_lines) > 5:
        return True
    else:
        return False

In [38]:
def clean_table(pandas_table, min_len, max_len):
    """ фильтруем таблицу """
    pandas_table["length"] = pandas_table.notes.apply(lambda x: len(x))
    pandas_table = pandas_table[(pandas_table["length"] < 1500) & (pandas_table["length"] > 500)]
    pandas_table["russian"] = pandas_table["notes"].apply(lambda x: is_russian(x))
    pandas_table = pandas_table[(pandas_table["russian"] == True)]
    pandas_table["poetry"] = pandas_table["notes"].apply(lambda x: is_poetry(x))
    pandas_table = pandas_table[(pandas_table["poetry"] == False)]
    return pandas_table

In [39]:
# загрузка дампа данных

dw = prozhito_tools.dump.Wrapper(csvpath='./prozhito_dump/')

In [40]:
# количество записей, количество авторов

len(dw.notes), len(dw.authors)

(384587, 3464)

### Создаём табличку с данными о наших дневниковых записях.

In [41]:
# собираем записи, их даты и айдишники дневников

all_notes = []
all_dates = []
all_ids = []
all_authors = []
for x in dw.notes:
    try:
        if x.diary:
            all_notes.append(x.text)
            all_dates.append("/".join([str(b) for b in x.date]))
            all_ids.append(x.diary_ID) 
        try:
            all_authors.append(x.diary.author.name)
        except AttributeError:
            all_authors.append("no author")
    except KeyError:
        continue

In [42]:
# делаем табличку

notes_pd = pd.DataFrame()

notes_pd["notes"] = all_notes
notes_pd["dates"] = all_dates
notes_pd["id"] = all_ids
notes_pd["author"] = all_authors

In [43]:
notes_pd.head()

,notes,dates,id,author
0,Воскресенье. \nВчера приехал Слава навестит...,0/0/0,2950,Василий Сергеевич Савельев
1,"Ночью мы достигли банки Антон-Дорн, в Датском ...",0/0/0,2268,Олег Владимирович Амитров
2,Суббота. Москва. Еду в Селище.,0/12/21,795,Сергей Иванович Танеев
3,18 Августа Его Королевское Величество отправил...,1609/8/18,1081,Самуил Бельский
4,"На следующий день, 19 Августа, прибыл в Остров...",1609/8/19,1081,Самуил Бельский


In [44]:
notes_pd["length"] = notes_pd.notes.apply(lambda x: len(x))

In [45]:
# среднее и медиана
np.mean(notes_pd["length"]), np.nanmedian(notes_pd["length"])

(1127.4357702730115, 564.0)

In [46]:
# перцентили
np.nanquantile(notes_pd["length"], 0.2), \
np.nanquantile(notes_pd["length"], 0.80)

(197.0, 1564.0)

In [47]:
# фильтруем табличку с записями нужной длины
cleaned_notes_pd = clean_table(notes_pd, 500, 1600)

/home/laidhimonthegreen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [48]:
notes_pd["poetry"] = notes_pd.notes.apply(lambda x: is_poetry(x))

In [49]:
cleaned_notes_pd.to_csv("big_cleaned_table.csv", sep="\t")

In [50]:
cleaned_notes_pd["note_id"] = cleaned_notes_pd.index

In [51]:
# записываем табличку
cleaned_notes_pd.to_csv("big_cleaned_table.csv", sep="\t")

In [15]:
cleaned_notes_pd["normalized"] = cleaned_notes_pd["notes"].apply(lambda x: normalize(x))

/home/laidhimonthegreen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [2]:
# формируем небольшие датасеты для разметки

In [25]:
cleaned_notes_pd.sample(1000).to_csv("sample_1-05.csv", sep="\t")

In [19]:
emotions_pd = cleaned_notes_pd[cleaned_notes_pd.notes.str.contains("не грустно") == True]
emotions_pd2 = cleaned_notes_pd[cleaned_notes_pd.notes.str.contains("тоскую") == True]
emotions_pd3 = cleaned_notes_pd[cleaned_notes_pd.notes.str.contains("счастлив,") == True]
emotions_pd4 = cleaned_notes_pd[cleaned_notes_pd.notes.str.contains("завидую") == True]
emotions_pd5 = cleaned_notes_pd[cleaned_notes_pd.notes.str.contains("люблю его") == True]
emotions_pd6 = cleaned_notes_pd[cleaned_notes_pd.notes.str.contains("одиноко") == True]

In [20]:
all_emotions_pd = pd.concat([emotions_pd, emotions_pd2, emotions_pd3, emotions_pd4, emotions_pd5, emotions_pd6])

In [21]:
len(all_emotions_pd)

2188

In [1]:
# pyotr_pd = cleaned_notes_pd[cleaned_notes_pd.author == "Петр Степанович Непорожний"]

In [22]:
all_emotions_pd.to_csv("emotions_pd.csv", sep = "\t")

In [23]:
work_pd = cleaned_notes_pd[cleaned_notes_pd.notes.str.contains(" доклад") == True]
work_pd2 = cleaned_notes_pd[cleaned_notes_pd.notes.str.contains(" основные задачи") == True]

In [24]:
works_pd = pd.concat([work_pd, work_pd2])

In [25]:
len(works_pd)

9390

In [28]:
works_pd.sample(1500).to_csv("works_pd.csv", sep = "\t")

In [14]:
cleaned_notes_pd["cyrillic"] = cleaned_notes_pd["notes"].apply(lambda x: is_cyrillic(x))

/home/laidhimonthegreen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
cleaned_notes_pd.head()

,notes,dates,id,author,length,cyrillic
0,Воскресенье. \nВчера приехал Слава навестит...,0/0/0,2950,Василий Сергеевич Савельев,313,True
1,"Ночью мы достигли банки Антон-Дорн, в Датском ...",0/0/0,2268,Олег Владимирович Амитров,731,True
3,18 Августа Его Королевское Величество отправил...,1609/8/18,1081,Самуил Бельский,155,True
7,"22, к ночи, дошел до Молодочны, куда прибыл ко...",1609/8/22,1081,Самуил Бельский,558,True
10,"25, в Минск, где его милость пан Гетман польны...",1609/8/25,1081,Самуил Бельский,782,True
